In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from torchvision import datasets, transforms, models


In [0]:
# Define my model
class LeNet(nn.Module):
    def __init__(self):
      super(LeNet, self).__init__()
      # Define the layers and its parameters
      self.conv1 = nn.Conv2d(1,20,5,1)
      self.conv2 = nn.Conv2d(20,50,3,1)
      self.conv3 = nn.Conv2d(50,64,3,1)
      self.linear1 = nn.Linear(3*3*64, 128)
      self.linear2 = nn.Linear(128,47)
      self.dropout1 = nn.Dropout(0.2)
      self.dropout2 = nn.Dropout(0.25)
        
    def forward(self,x):
      # define the forward pass 
      out = F.relu(self.conv1(x))
      out = F.max_pool2d(out, 2, 2)
      out = self.dropout1(out)
      out = F.relu(self.conv2(out))
      out = F.max_pool2d(out, 2, 2)
      out = F.relu(self.conv3(out))
      out = self.dropout1(out)
      out = out.view(-1, 3*3*64)
      out = F.relu(self.linear1(out))
      out = self.dropout2(out)
      out = self.linear2(out)
      return F.log_softmax(out, dim=1)


In [0]:
# Define the training function
def train(model, device, train_data, optimizer, epoch):
    model.train() # enable the train model
    for batch_idx, (data, target) in enumerate(train_data):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # check the progress and the loss
        if batch_idx % 2000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx*len(data), len(train_loader.dataset), 
                100. * batch_idx / len(train_loader), loss.item()))
        

In [0]:
# Definr the testing function
def test(model, device, test_data):
    model.eval()
    # variables used to store the info about accuracy
    test_loss = 0 # loss
    correct_1 = 0 # top 1 correct number
    correct_5 = 0 # top 5 correct number
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # top 1 accuracy
            pred_1 = output.argmax(dim=1, keepdim=True)
            correct_1 += pred_1.eq(target.view_as(pred_1)).sum().item()
            # top 5 accuracy
            _,pred_5 = output.topk(5, dim=1)
            pred_5 = pred_5.t()
            correct = pred_5.eq(target.view(1, -1).expand_as(pred_5))
            correct_5 += correct[:5].view(-1).float().sum(0)
    test_loss /= len(test_loader.dataset)
    
    print('Test set: Average loss: {:.4f}, Top1 Accuracy: {}/{} ({:.0f}%), Top5 Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct_1, len(test_loader.dataset), 100. * correct_1 / len(test_loader.dataset), 
                   correct_5, len(test_loader.dataset), 100. * correct_5 / len(test_loader.dataset)))


In [0]:
# load the data set (train and test)
train_data = datasets.EMNIST('../data', split='bymerge', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.EMNIST('../data', split='bymerge', train=False, download=True, transform=transforms.ToTensor())


In [0]:
# transform it to DataLoader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
                                           shuffle=True, num_workers=1, pin_memory=True)
    
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32,
                                          shuffle=True, num_workers=1, pin_memory=True)


In [40]:
device = torch.device("cuda")
model = LeNet()
model.to(device)

# # define the model (transfer learning)
# model = models.vgg16_bn(pretrained=True)

# for param in model.parameters():
#   param.requires_grad = False

# # change the first layer
# new_features = nn.Sequential(*list(model.features.children()))
# new_features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
# new_features[0].weight.data.normal_(0, 0.001)
# model.features = new_features


# # last few layers
# fc = nn.Sequential(
#     nn.Linear(4096, 256),
#     nn.ReLU(),
#     nn.Dropout(0.4),
    
#     nn.Linear(256,47),
#     nn.LogSoftmax(dim=1)
    
# )

# # replace
# model.classifier[6] = fc


# # to GPU
# model.to(device)


LeNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(50, 64, kernel_size=(3, 3), stride=(1, 1))
  (linear1): Linear(in_features=576, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=47, bias=True)
  (dropout1): Dropout(p=0.2)
  (dropout2): Dropout(p=0.25)
)

In [0]:
# Declear the parameters
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
n_epoch = 5


In [42]:
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

118,353 total parameters.
118,353 training parameters.


In [44]:
# actual training and testing loop
for epoch in range(n_epoch):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    

Train Epoch: 1 [0/697932 (0%)]	Loss: 0.666445
Train Epoch: 1 [64000/697932 (9%)]	Loss: 0.297790
Train Epoch: 1 [128000/697932 (18%)]	Loss: 0.331214
Train Epoch: 1 [192000/697932 (28%)]	Loss: 0.550682
Train Epoch: 1 [256000/697932 (37%)]	Loss: 0.416740
Train Epoch: 1 [320000/697932 (46%)]	Loss: 0.533418
Train Epoch: 1 [384000/697932 (55%)]	Loss: 0.600635
Train Epoch: 1 [448000/697932 (64%)]	Loss: 0.678653
Train Epoch: 1 [512000/697932 (73%)]	Loss: 0.586725
Train Epoch: 1 [576000/697932 (83%)]	Loss: 0.636995
Train Epoch: 1 [640000/697932 (92%)]	Loss: 0.445142

Test set: Average loss: 0.2880, Top1 Accuracy: 103985/116323 (89%), Top5 Accuracy: 115952.0/116323 (100%)
Train Epoch: 2 [0/697932 (0%)]	Loss: 0.331670
Train Epoch: 2 [64000/697932 (9%)]	Loss: 0.153319
Train Epoch: 2 [128000/697932 (18%)]	Loss: 0.440465
Train Epoch: 2 [192000/697932 (28%)]	Loss: 0.254433
Train Epoch: 2 [256000/697932 (37%)]	Loss: 0.323875
Train Epoch: 2 [320000/697932 (46%)]	Loss: 0.561642
Train Epoch: 2 [384000/69

In [45]:
# mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#save the model
model_name = 'EMNISTModel.pt'
path = F"/content/gdrive/My Drive/{model_name}" 
torch.save(model.state_dict(), path)
